In [1]:
from difflib import get_close_matches
import pandas as pd
from mobfot import MobFot
from icecream import ic
from tqdm import tqdm
client  = MobFot()

def parse_time(time):
    return f"{time.replace('T', ' ').rsplit('.', 1)[0]}"


ids = list()
for x in client.get_league(77)['matches']['allMatches']:
    ids.append(x['id'])


df = pd.read_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv")
df['match_id'] = None

home_teams = df.home_team.unique()
away_teams = df.away_team.unique()
for id in tqdm(ids):
    match = client.get_match_details(id)['general']
    parsed_time = parse_time(match['matchTimeUTCDate'])
    date_string = parsed_time.split(' ')[0]
    time_string = parsed_time.split(' ')[1]
    time_string = str((int(time_string.split(':')[0]) + 3) % 24) + ':00:00'
    time = date_string + ' ' + time_string
    home_team = match['homeTeam']['name']
    if home_team == 'USA':
        home_team = 'United States'
    if home_team == 'South Korea':
        home_team = 'Korea Republic'
    home_team = get_close_matches(home_team, home_teams, n=1, cutoff=0.5)[0]
    
    away_team = match['awayTeam']['name']
    if away_team == 'USA':
        away_team = 'United States'
    if away_team == 'South Korea':
        away_team = 'Korea Republic'
    away_team = get_close_matches(away_team, away_teams, n=1, cutoff=0.5)[0]
    # print(f"Currently adding {time} {home_team} vs {away_team}")
    df.loc[(df.match_time == time) & (df.home_team == home_team) & (df.away_team == away_team), 'match_id'] = id

df.to_csv("FIFA DataSet/Data/FIFA World Cup 2022 Match Data/data.csv", index=False)


100%|██████████| 64/64 [00:32<00:00,  1.98it/s]


In [2]:
# first match of the Netherlands
client.get_team('613792')

In [4]:
# Spain costa rica
client.get_team('3854585')

In [11]:


def get_player_data(match):
    homePlayers = list()
    for x in match['content']['lineup']['lineup'][0]['players']:
        for y in x:
            homePlayers.append(y)

    awayPlayers = list()
    for x in match['content']['lineup']['lineup'][1]['players']:
        for y in x:
            awayPlayers.append(y)
            
    return homePlayers, awayPlayers
            

def get_player_by_name(name, players):
    allPlayers = [x['name']['lastName'] for x in players]
    try:

        match = get_close_matches(name, allPlayers, n=1, cutoff=0.5)[0]
        return players[allPlayers.index(match)]
    except:
        return None
    



In [38]:

id = 3854579
match = client.get_match_details(id)

homePlayers, awayPlayers = get_player_data(match)

olmo = get_player_by_name('Olmo', homePlayers)

In [30]:
ic(olmo['shotmap'][0])

ic| olmo['shotmap'][0]: {'blockedX': None,
                         'blockedY': None,
                         'eventType': 'Miss',
                         'expectedGoals': 0.372837632894516,
                         'expectedGoalsOnTarget': None,
                         'goalCrossedY': 42.061562499999994,
                         'goalCrossedZ': 0.2696842092,
                         'id': 2487365161,
                         'isBlocked': False,
                         'isOnTarget': False,
                         'isOwnGoal': False,
                         'isSavedOffLine': False,
                         'min': 5,
                         'minAdded': None,
                         '

onGoalShot': {'x': 0,
                                        'y': 0.03345309413652753,
                                        'zoomRatio': 0.46889173159669767},
                         'period': 'FirstHalf',
                         'playerId': 614834,
                         'playerName': 'Dani Olmo',
                         'shotType': 'RightFoot',
                         'situation': 'RegularPlay',
                         'teamColor': '#c62035',
                         'teamId': 6720,
                         'x': 98.6315789471,
                         'y': 28.707142856}


{'id': 2487365161,
 'eventType': 'Miss',
 'teamId': 6720,
 'playerId': 614834,
 'playerName': 'Dani Olmo',
 'x': 98.6315789471,
 'y': 28.707142856,
 'min': 5,
 'minAdded': None,
 'isBlocked': False,
 'isOnTarget': False,
 'blockedX': None,
 'blockedY': None,
 'goalCrossedY': 42.061562499999994,
 'goalCrossedZ': 0.2696842092,
 'expectedGoals': 0.372837632894516,
 'expectedGoalsOnTarget': None,
 'shotType': 'RightFoot',
 'situation': 'RegularPlay',
 'period': 'FirstHalf',
 'isOwnGoal': False,
 'onGoalShot': {'x': 0,
  'y': 0.03345309413652753,
  'zoomRatio': 0.46889173159669767},
 'isSavedOffLine': False,
 'teamColor': '#c62035'}

In [41]:
for player in homePlayers:
    
    for shot in player['shotmap']:
        print(f"Shot from {round(shot['x'],2)}, {round(shot['y'], 2)}")
        print(f"Shot in {shot['period']}")
        if shot['isOnTarget']:
            print(f"On target at {round(shot['goalCrossedY'],2)}, {round(shot['goalCrossedZ'],2)}")
        else:
            print(f"Miss at {round(shot['goalCrossedY'],2)}, {round(shot['goalCrossedZ'],2)}")
        print("\n")
        

Shot from 98.54, 35.14
Shot in SecondHalf
Miss at 22.11, 1.16


Shot from 98.63, 32.02
Shot in SecondHalf
On target at 32.32, 1.18


Shot from 82.6, 33.01
Shot in SecondHalf
On target at 33.08, 1.22


Shot from 96.8, 30.95
Shot in FirstHalf
On target at 34.76, 0.98


Shot from 81.02, 26.63
Shot in FirstHalf
On target at 34.84, 0.62


Shot from 92.3, 35.83
Shot in FirstHalf
Miss at 18.11, 3.74


Shot from 99.02, 21.83
Shot in FirstHalf
On target at 33.08, 1.22


Shot from 85.17, 26.86
Shot in FirstHalf
Miss at 29.21, 0.44


Shot from 91.1, 36.75
Shot in SecondHalf
On target at 31.33, 1.22


Shot from 90.4, 26.69
Shot in FirstHalf
Miss at 27.78, 2.55


Shot from 98.34, 33.62
Shot in FirstHalf
Miss at 31.79, 4.22


Shot from 95.06, 40.64
Shot in SecondHalf
On target at 32.02, 1.22


Shot from 94.29, 40.97
Shot in SecondHalf
On target at 32.55, 1.22


Shot from 93.8, 47.43
Shot in SecondHalf
On target at 32.86, 1.22


Shot from 101.02, 47.64
Shot in SecondHalf
Miss at 0.66, 0.09


Shot fro

In [29]:
# 105x68

68/2

34.0

In [37]:
olmo

{'id': '614834',
 'positionId': 107,
 'position': 'Attacker',
 'positionStringShort': 'LW',
 'localizedPosition': {'label': 'LW', 'key': 'leftwinger_short'},
 'name': {'firstName': 'Dani', 'lastName': 'Olmo', 'fullName': 'Dani Olmo'},
 'shirt': 21,
 'usualPosition': 3,
 'usingOptaId': False,
 'teamId': '6720',
 'imageUrl': 'https://images.fotmob.com/image_resources/playerimages/614834.png',
 'pageUrl': '/players/614834/dani-olmo',
 'isHomeTeam': True,
 'timeSubbedOn': None,
 'timeSubbedOff': None,
 'positionRow': 5,
 'role': 'Attacker',
 'isCaptain': False,
 'events': {'g': 1, 'as': 1},
 'rating': {'num': '8.7',
  'bgcolor': '#1ec853',
  'isTop': {'isMatchFinished': True}},
 'fantasyScore': {'num': None, 'bgcolor': 'purple'},
 'minutesPlayed': 90,
 'shotmap': [{'id': 2487365161,
   'eventType': 'Miss',
   'teamId': 6720,
   'playerId': 614834,
   'playerName': 'Dani Olmo',
   'x': 98.6315789471,
   'y': 28.707142856,
   'min': 5,
   'minAdded': None,
   'isBlocked': False,
   'isOnTarg

In [ ]:
# Goal 2.44x7.32